In [34]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'browser'
from urllib.request import urlopen
import json
from copy import deepcopy
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Creating one dataframe for all stores

In [24]:
# Combine csv files into one dataframe

# List of CSV file paths
csv_files = ["../data/processed/aldi_data.csv", "../data/processed/coop_data.csv", "../data/processed/lidl_data.csv", "../data/processed/migros_data.csv"]

# Create an empty DataFrame to store the combined data
stores_df = pd.DataFrame()

# Iterate through each CSV file and concatenate the data
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    
    # Add a new column with the original DataFrame name
    df["name"] = csv_file
    
    stores_df = pd.concat([stores_df, df], ignore_index=True)

# Display the combined DataFrame
stores_df

,address,latitude,longitude,kreis_name,name
0,"Stadelhoferstrasse 10, 8001",47.366789,8.548001,Kreis 1,../data/processed/aldi_data.csv
1,"Mythenquai 353, 8038",47.345470,8.534889,Kreis 2,../data/processed/aldi_data.csv
2,"Albisstrasse 81, 8038",47.340868,8.530663,Kreis 2,../data/processed/aldi_data.csv
3,"Witikonerstrasse 279, 8053",47.358731,8.585599,Kreis 7,../data/processed/aldi_data.csv
4,"Sihlstrasse 3, 8001",47.373727,8.537653,Kreis 1,../data/processed/aldi_data.csv
...,...,...,...,...,...
109,"Hardturmstrasse 11, 8005",47.391344,8.518489,Kreis 5,../data/processed/migros_data.csv
110,"Tessinerplatz 10, 8002",47.364369,8.530935,Kreis 2,../data/processed/migros_data.csv
111,"Birmensdorferstrasse 320, 8055",47.369893,8.508304,Kreis 3,../data/processed/migros_data.csv
112,"Scheffelstrasse 3, 8037",47.393773,8.529234,Kreis 10,../data/processed/migros_data.csv


In [25]:
# Cleaning the name column
# Extract store name from 'name' column
stores_df["store"] = stores_df["name"].apply(lambda x: x.split("/")[1].split("_")[0])

In [26]:
# Sorting columns
column_order = ["store"] + [col for col in stores_df.columns if col != "store"]
stores_df = stores_df[column_order]

In [27]:
# Drop the "name" column
stores_df = stores_df.drop("name", axis=1)

In [28]:
# Cleaning kreis_name column - turning string to int
stores_df["Kreis"] = stores_df["kreis_name"].str.extract(r'Kreis (\d+)')
stores_df = stores_df.drop("kreis_name", axis=1)
stores_df["Kreis"] = stores_df["Kreis"].astype(int)
stores_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   store      114 non-null    object 
 1   address    114 non-null    object 
 2   latitude   114 non-null    float64
 3   longitude  114 non-null    float64
 4   Kreis      114 non-null    int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 4.6+ KB


In [29]:
# Adding Kreis info

df_kreis = pd.read_csv("../data/processed/Kreis_info.csv")
all_stores_df = pd.merge(stores_df, df_kreis, on="Kreis", how="inner")
all_stores_df

,store,address,latitude,longitude,Kreis,Kreis Area (km^2),Kreis Population,Kreis Density (population/km^2)
0,data,"Stadelhoferstrasse 10, 8001",47.366789,8.548001,1,1.80,5817,3232
1,data,"Sihlstrasse 3, 8001",47.373727,8.537653,1,1.80,5817,3232
2,data,"Bahnhofbrücke 1, 8001",47.376458,8.542357,1,1.80,5817,3232
3,data,"Halle Löwenstrasse, 8021",47.378042,8.539067,1,1.80,5817,3232
4,data,"Stadelhoferstrasse 10, 8001",47.366789,8.548001,1,1.80,5817,3232
...,...,...,...,...,...,...,...,...
109,data,"Seefeldstrasse 123, 8008",47.358377,8.555105,8,4.81,17818,3704
110,data,"Seefeldstrasse 35, 8008",47.363175,8.549919,8,4.81,17818,3704
111,data,"Forchstrasse 4, 8008",47.364443,8.554834,8,4.81,17818,3704
112,data,"Seefeldstrasse 111 - 113, 8008",47.358365,8.554159,8,4.81,17818,3704


In [30]:
# Save DataFrame to CSV
all_stores_df.to_csv('all_stores_data.csv', index=False)

### Plotting data

In [31]:
# Copy the dataset
df_stores = deepcopy(all_stores_df)

In [32]:
with open("../data/raw/stzh.adm_stadtkreise_a.json") as response:
    kreise = json.load(response)

In [35]:
# Get unique store names and assign a color to each
unique_stores = df_stores['store'].unique()
store_colors = {
    'aldi': 'blue',
    'coop': 'red',
    'lidl': 'yellow',
    'migros': 'orange'
}

# Create a scatter map for store locations
scatter_map = go.Scattermapbox(
    lat=df_stores['latitude'],
    lon=df_stores['longitude'],
    mode='markers',
    marker=dict(
        size=10,
        color=df_stores['store'].map(store_colors)
    ),
    text=df_stores['store'] + '<br>' + df_stores['address']  # Display store brand and address on hover
)

# Create a choropleth map for district density
choropleth_map = go.Choroplethmapbox(
    geojson=kreise,
    locations=df_stores['Kreis'],
    featureidkey="properties.name",
    z=df_stores['Kreis Density (population/km^2)'],
    colorscale='Blues',  # Choose a colorscale
    colorbar=dict(title='District Density')
)

# Create a layout for the map
layout = go.Layout(
    mapbox=dict(
        center={"lat": 47.38, "lon": 8.54},
        style="carto-positron",
        zoom=10,
    )
)

# Create a Figure with both scatter and choropleth maps
fig = go.Figure(data=[scatter_map, choropleth_map], layout=layout)

# Show the map
fig.show()